In [1]:
import pandas as pd
import sqlite3

## 1

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## 2

In [9]:
try:
    pd.io.sql.read_sql(
        """
        CREATE TABLE IF NOT EXISTS datamart AS
            SELECT c.uid, c.labname, c.timestamp AS first_commit_ts, pg.first_view_ts
            FROM checker AS c LEFT JOIN (
                SELECT uid, MIN(datetime) AS first_view_ts
                FROM pageviews
                WHERE uid LIKE "user_%"
                GROUP BY uid) AS pg 
            ON c.uid = pg.uid
            WHERE status = "ready" AND numTrials = 1 
                  AND labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s", "project1")
                  AND c.uid LIKE "user_%"
            ORDER BY first_commit_ts;
        """,
        con=conn
    )
except TypeError:
    "Таблица создалась"

In [12]:
pd.io.sql.read_sql(
    """
    SELECT * FROM datamart
    ORDER BY uid;
    """,
    con=conn
)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
4,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,None
136,user_8,laba05,2020-05-02 13:28:07.705193,None
137,user_8,project1,2020-05-14 15:42:04.002981,None
138,user_8,laba06,2020-05-16 17:56:15.755553,None


## 3

In [13]:
test = pd.io.sql.read_sql(
    """
    SELECT * FROM datamart
    WHERE first_view_ts NOT NULL;
    """,
    con=conn
)
test['first_view_ts'] = pd.to_datetime(test['first_view_ts'])

In [15]:
test.head(10)

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [18]:
test.to_sql('test', con=conn)

59

In [19]:
control = pd.io.sql.read_sql(
    """
    SELECT * FROM datamart
    WHERE first_view_ts IS NULL;
    """,
    con=conn
)
control['first_view_ts'] = test['first_view_ts'].mean()

In [20]:
control.head(10)

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
3,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
4,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552
5,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
6,user_8,laba04,2020-04-18 21:53:36.058389,2020-04-27 00:40:05.761783552
7,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783552
8,user_29,laba04,2020-04-19 11:10:55.722647,2020-04-27 00:40:05.761783552
9,user_29,laba04s,2020-04-19 18:55:41.516438,2020-04-27 00:40:05.761783552


In [22]:
control.to_sql('control', con=conn)

81

## 4

In [23]:
conn.close()